# Likelihood functions in `hssm` explained

One of the design goals of `hssm` is its flexibility. It is built from ground up to support many types of likelihood functions out-of-the-box. For more sophisticated use cases, `hssm` provides a convenient toolbox to help the user build their own likelihood functions that can be used with `hssm`. This notebook focuses on explaining how to use different types of likelihoods with `hssm`.

## 3 Kinds of Likelihoods

`hssm` supports 3 kinds of likelihood functions supported via the `loglik_kind` parameter to the `HSSM` class:

- `"analytical"`: These likelihoods are usually closed-form solutions to the actual likelihoods. For example, For `ddm` models, `hssm` provides the analytical likelihoods in Navarro & Fuss (2009). These likelihoods are typically Python functions written with `pytensor`, and thus can be compiled by `pytensor` as part of a computational graph. As such, they are differentiable as well.
- `"approx_differentiable"`: These likelihoods are usually approximations of the actual likelihood functions with neural networks. These networks can be trained with any popular deep learning framework such as `PyTorch` and `TensorFlow` and saved as `onnx` files. `hssm` can load the `onnx` files and translate the information of the neural network with either the `jax` or the `pytensor` backends. Please see below for detailed explanations for these backends. The `backend` option can be supplied via the `"backend"` field via `model_config`. This field of `model_config` is not applicable to other kinds of likelihoods.

    - the `jax` backend: the feed-forward and back-propagation processes are computed with "JAX", which is wrapped in a `pytensor` `Op`. When sampling using the default NUTS sampler in `PyMC`, this option might be slightly faster but more prone to compatibility issues especially during parallel sampling due how `JAX` handles paralellism. When sampling using a JAX based sampler such as `nuts_numpyro` and `black_jax`, the `JAX` computation will be taken out of the `Op` and compiled together with the rest of the computation graph in `JAX`. Therefore, if a `JAX`-based sampler is used, this is a better option.
    - the `pytensor` backend: the feed-forward and back-propagation processes are computed with `pytensor`. When sampling using the default NUTS sampler in `PyMC`, this option allows for maximum compatibility. Not recommended when using `JAX`-based samplers.
 
- `"blackbox"`: Use this option for "black box" likelihoods that are not differentiable. These likelihoods are typically `Callable`s in Python that cannot be directly integrated to a `pytensor` computational graph. `hssm` will wrap these `Callable`s in a `pytensor` `Op` so it can be part of the graph.

## Default vs. Custom Likelihoods

`hssm` provides many default likelihood functions out-of-the-box. The supported likelihoods are:

- For `analytical` kind: `ddm` and `ddm_sdv` models.
- For 